# Preparing the dataset
This notebook describes how to create a timeseries dataset for use with the CNTK minibatch source. 
The dataset for this sample is a [free open-source dataset](https://www.cntk.ai/jup/dat/solar.csv) containing measurements of a set of solar panels during the day. The data is stored as a CSV file on disk, so we can use pandas to process it.

The output of this notebook is a CTF file containing sequences of varying length used to train a recurrent neural network to predict total solar power output for a set of solar panels. We'll produce two datasets: A training set and a validation set.

## Loading the data
The dataset is stored as a table not a set of sequences. First we'll need to load the data and normalize it so we have proper input to generate sequences from. The dataset has a timestamp we can use as the index. This makes it easier to group the data per day so we can generate sequences for a specific day.

In [42]:
import pandas as pd
import numpy as np

In [62]:
df_solar = pd.read_csv('solar.csv', index_col='time', parse_dates=['time'])

df_solar['date'] = df_solar.index.date

print(df_solar['solar.total'].max())

# Normalize the data so all values are between 0 and 1.
# This is required, because we are using sigmoid and tanh activations in our model.
# These kind of activations don't work for values that are not within the 0 to 1 range.
df_solar['solar.current'] /= df_solar['solar.total'].max()
df_solar['solar.total'] /= df_solar['solar.total'].max()

19100.0


The result of the code above is that we now have a dataset that has an index containing the timestamps for the measurements. The dataset contains normalized values for the current output and the total output for a day. We can now start to group up measurements per day and calculate the total power generated for each day.

In [44]:
df_grouped = df_solar.groupby(df_solar.index.date).max()
df_grouped.columns = ['solar.current.max', 'solar.total.max', 'date']

The grouped dataset contains the total power generated for a particular day `solar.total.max`. It also contains the maximum power generated in 30 minutes for that day which is stored in `solar.current.max`. We can now merge both datasets to get a dataset that contains the original sequences, but with the totals for each day added to each entry of the sequence.

In [45]:
df_merged = pd.merge(df_solar, df_grouped, right_index=True, on='date')
df_merged = df_merged[['solar.current', 'solar.total', 'solar.current.max','solar.total.max']]

df_per_day = df_merged.groupby(df_merged.index.date)

## Preprocessing the data
The data is stored as a table but we need to get sequences into a CTF file. We're going to have to create sequences from the original dataset. Each day is its own sequence that we can use to predict the total power generated for a day.

There are a few things that we have to keep in mind to ensure that our model does sensible things:

 * Each day that has less than 8 measurements is considered faulty and discarded.
 * Each day that has more than 14 measurements is truncated to 14 measurements.
 
We'll create two lists of datapoints, one with the targets for each day and another one that contains the sequence of datapoints for that day. 

In [47]:
targets = []
sequences = []

for _, group in df_per_day:
    # Less then 8 measurements on a day is considered invalid.
    if len(group['solar.total'].values) < 8:
        continue
        
    day_total = group['solar.total.max'].values[0]
    sequence = group[['solar.total']].values[0:14, :] 
    
    for j in range(2, len(sequence)):
        derived_seq = sequence[:j]
        sequences.append(derived_seq)
        targets.append(day_total)

## Storing the data
Once we have the data preprocessed into sequences and targets, let's create the CTF file. The CTF file format says that you can store a sequence over multiple lines, so each sample from the sequence goes on a separate line to make things simple.
This looks like this:

```
0 |target 0.5392670157068062 |features 8.848167838850571e-05
0 |features 0.000594764392413394
1 |target 0.5392670157068062 |features 8.848167838850571e-05
1 |features 0.000594764392413394
1 |features 0.0035340314136125656
2 |target 0.5392670157068062 |features 8.848167838850571e-05
2 |features 0.000594764392413394
2 |features 0.0035340314136125656
2 |features 0.013115183246073298
```
The first line of a new sequence includes the target for that sequence.

To properly train the model we need to have two datasets, a training set and a validation set.
We're splitting the whole dataset in three chunks:

 1. A training set containing 70% of all the data.
 2. A validation set containing 20% of all the data.
 3. A test set containing 10% of all the data.
 
We'll store the first two sets in a CTF file format for use with a minibatch source later on. We're going to store the test set as a pickle file with numpy arrays. This makes it easier to load the test samples in a ready-to-go format for making predictions with our model later on.

In [66]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sequences,targets, test_size=0.1)
train_x, val_x, train_y, val_y = train_test_split(X_train, y_train, test_size=0.2)

In [67]:
def store_dataset(filename, x, y):
    with open(filename, 'w') as output_file:
        for i in range(0, len(y)):
            sequence = x[i]
            target = y[i]

            for j,element in enumerate(sequence):
                output_file.write('{} '.format(i))

                if j == 0:
                    output_file.write('|target {} '.format(target))

                output_file.write('|features {}\n'.format(element[0]))    

In [68]:
store_dataset('solar_train.ctf', train_x, train_y)
store_dataset('solar_val.ctf', val_x, val_y)

In [76]:
import pickle

test_items = []

for item in X_test:
    test_items.append(np.array(item))
    
with open('test_samples.pkl', 'wb') as test_file:
    pickle.dump(test_items, test_file)